In [ ]:
from keras.models import model_from_yaml
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

In [1]:
from PIL import Image
from skimage import io

import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.externals import joblib
import numpy as np

class PathFinder: 
    def __init__(self,model_source,video_source='test2.mp4'):
        self.clf = joblib.load(model_source)
        self.cap=cv2.VideoCapture(video_source)
            
    
    def StartRide(self):
        frame=self.cap.read()[1]
        while(self.cap.isOpened()):
            frame=self.cap.read()[1]
            frame = cv2.resize(frame,(640, 380), interpolation = cv2.INTER_CUBIC)
            #frame = cv2.GaussianBlur(frame,(3,3),0)
            predict=self.__detectSigns(frame)
            cv2.imshow('frame2',frame)
            if cv2.waitKey(2) & 0xFF == ord("q"):
                break     
        self.cap.release()
        cv2.destroyAllWindows()
    
    def __detectSigns(self,frame):
        pred = []
        low_red = (0,0,200)
        hight_red = (140,140,255)
        mask = cv2.inRange(frame, low_red, hight_red)
        cont = cv2.findContours(mask,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)[1]
        for cnt in cont:
            moments = cv2.moments(cnt, 255)
            dM01 = moments['m01']
            dM10 = moments['m10']
            dArea = moments['m00']
            if dArea > 30:                #Отбрасывание контуров по площади
                x = int(dM10 / dArea) # x координата центра масс контура
                y = int(dM01 / dArea) # у координата цента масс контура
                #L,R,T,B - кайние точки контура, левая, правая, верхняя, нижняя соответственно
                L = tuple(cnt[cnt[:, :, 0].argmin()][0])
                R = tuple(cnt[cnt[:, :, 0].argmax()][0])
                T = tuple(cnt[cnt[:, :, 1].argmin()][0])
                B = tuple(cnt[cnt[:, :, 1].argmax()][0])
                diam1 = R[0] - L[0]
                diam2 = B[1] - T[1]
                r = int((diam1+diam2)/4) # средний радиус контура
                if(y > 3*r and x > 3*r and diam1/diam2 > 0.85 and diam1/diam2 < 1.15): # первые 2 условия для корректного вырезания контура
                    cut_frame = []
                    if r < 5: # С этими условиями результат получается лучше
                        cut_frame = frame[(y-r*3):(y+r*9),(x-r*3):(x+r*3)]
                    else:
                        cut_frame = frame[(y-r*2):(y+r*7),(x-r*2):(x+r*2)]
                    cut_frame = cv2.resize(cut_frame,(16, 48), interpolation = cv2.INTER_CUBIC)
                    hog=self.__hog(cut_frame,8)
                    if self.clf.predict_proba([hog])[0][1] > 0.8:
                        pred = cv2.rectangle(frame,(x-r*3,y-r*3),(x+r*3,y+r*8),(0,0,255),2)
        return pred

In [2]:
asdf = PathFinder('D:model_semafors.pkl')
asdf.StartRide()

FileNotFoundError: [Errno 2] No such file or directory: 'D:model_semafors.pkl'